🚌 Projet MDM - Mobilité Durable en Montagne ⛰️

*Author : Nicolas Grosjean*

*Date : 09/11/2025*

**Description :**

Analyses bus data with C2C activity points :
- Compute distance between them
- Identify C2C activity points not deserved
- Make pretty visualisations to share

In [1]:
# WORKING DIR NEED TO BE SET BEFORE IMPORTING SETTINGS
import os

os.chdir("../..")
print("Working directory set to the root of the project")

Working directory set to the root of the project


In [2]:
import folium
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, box

from src.settings import EPSG_WEB_MERCATOR, EPSG_WGS84

In [3]:
isere_code = "38"

## Read data

In [4]:
from src.processors.osm import OSMBusStopsProcessor

In [5]:
osm_stops_gdf = OSMBusStopsProcessor.fetch(reload_pipeline=False).set_crs(
    EPSG_WGS84
)  # TODO Remove CRS
osm_stops_gdf.loc[:, "source"] = "OSM"
osm_stops_gdf.head()

,gtfs_id,navitia_id,osm_id,name,description,line_gtfs_ids,line_osm_ids,network,network_gtfs_id,geometry,other,source
0,None,None,135296,Université - IUT-STAPS,None,[],[3922428],M réso,None,POINT (5.77622 45.19738),"{'alt_name': None, 'amenity': None, 'bench': N...",OSM
1,None,None,135930,Hôpital Couple Enfant,None,[],"[3333927, 3922430]",M réso,None,POINT (5.74231 45.20065),"{'alt_name': None, 'amenity': None, 'bench': N...",OSM
2,None,None,136570,Cap des H',"Arrêt de régulation, non commercial.",[],[],M réso,None,POINT (5.68159 45.21695),"{'alt_name': None, 'amenity': None, 'bench': N...",OSM
3,None,None,136597,Place de la Libération,None,[],"[3031947, 3031948, 3044780, 3044781, 3923550, ...",M réso,None,POINT (5.66272 45.20707),"{'alt_name': None, 'amenity': None, 'bench': N...",OSM
4,None,None,137073,Centr'Alp 2,None,[],"[8671286, 8671287]",M réso,None,POINT (5.6043 45.3198),"{'alt_name': None, 'amenity': None, 'bench': N...",OSM


In [6]:
activity_gdf = gpd.read_parquet("src/data_2/C2C/depart_topos_stops_isere.parquet")
print(f"{len(activity_gdf)} activity points")
activity_gdf.rename(
    columns={"navitia_id": "Id wp", "name": "Name wp", "nombre_de_depart_de_topo": "nbr_topo"},
    inplace=True,
)
activity_gdf = activity_gdf[["Id wp", "Name wp", "geometry", "nbr_topo"]]
activity_gdf.head()

577 activity points


,Id wp,Name wp,geometry,nbr_topo
0,38440,Pierre Blanche,POINT (5.52399 44.88944),3
1,39041,Alpe d'Huez - Falaises du lac Besson,POINT (6.09444 45.11727),5
2,39108,Les Arias d'en bas (Mariande),POINT (6.17616 44.91703),1
3,39178,Cirque inférieur du Boulon,POINT (5.96136 45.19128),1
4,39195,Le Bourg-d'Oisans - Commères,POINT (6.08117 45.0272),11


In [7]:
area_gdf = gpd.read_file("src/data/transportdatagouv/contour-des-departements.geojson")
area_gdf = area_gdf.set_crs(EPSG_WGS84, allow_override=True)
area_gdf.head()

,code,nom,geometry
0,01,Ain,"POLYGON ((4.78021 46.17668, 4.78024 46.18905, ..."
1,02,Aisne,"POLYGON ((3.17296 50.01131, 3.17382 50.01186, ..."
2,03,Allier,"POLYGON ((3.03207 46.79491, 3.03424 46.7908, 3..."
3,04,Alpes-de-Haute-Provence,"POLYGON ((5.67604 44.19143, 5.67817 44.19051, ..."
4,05,Hautes-Alpes,"POLYGON ((6.26057 45.12685, 6.26417 45.12641, ..."


In [8]:
isere_activity_df = (
    gpd.sjoin(
        activity_gdf.to_crs(EPSG_WGS84),
        area_gdf[area_gdf["code"] == isere_code].to_crs(EPSG_WGS84),
    )
    .loc[:, ["Id wp"]]
    .drop_duplicates()
    .merge(activity_gdf, on="Id wp", how="inner")
)
isere_activity_gdf = gpd.GeoDataFrame(isere_activity_df, geometry="geometry").set_crs(
    EPSG_WGS84
)
print(f"Filter on Isère: keep {len(isere_activity_gdf)} on {len(activity_gdf)}")

Filter on Isère: keep 514 on 577


In [9]:
tdg_stops_gdf = gpd.read_parquet(
    os.path.join(os.getcwd(), "src/data/transportdatagouv/stops_38.parquet")
)
tdg_stops_gdf.columns = [
    "network_gtfs_id",
    "network",
    "gtfs_id",
    "name",
    "stop_code",
    "description",
    "line_gtfs_ids",
    "geometry",
]
tdg_stops_gdf.loc[:, "source"] = "TDG"
tdg_stops_gdf.head()

,network_gtfs_id,network,gtfs_id,name,stop_code,description,line_gtfs_ids,geometry,source
0,CARSxREGIONxAIN:Network:1:LOC,REGION - cars Région Ain,FR:38261:ZE:38367:CARSxREGIONxAIN,La gare,1136154,<NA>,{'line_id': 'CARSxREGIONxAIN:FlexibleLine:1005...,POINT (608999.29 5727934.37),TDG
1,CARSxREGIONxAIN:Network:1:LOC,REGION - cars Région Ain,FR:38055:ZE:38540:CARSxREGIONxAIN,Mairie,1137588,<NA>,{'line_id': 'CARSxREGIONxAIN:FlexibleLine:1005...,POINT (615432.365 5731543.476),TDG
2,CARSxREGIONxAIN:Network:1:LOC,REGION - cars Région Ain,FR:38465:ZE:38541:CARSxREGIONxAIN,Place,1137590,<NA>,{'line_id': 'CARSxREGIONxAIN:FlexibleLine:1005...,POINT (612596.334 5731950.939),TDG
3,ARDECHE:Network:1:LOC,REGION - cars Région Ardèche,FR:38298:ZE:40822:ARDECHE,Gare SNCF,1023073,<NA>,"{'line_id': 'ARDECHE:Line:1000601:LOC', 'line_...",POINT (533823.813 5680267.662),TDG
4,ARDECHE:Network:1:LOC,REGION - cars Région Ardèche,FR:38349:ZE:41169:ARDECHE,Mairie,1027195,<NA>,"{'line_id': 'ARDECHE:Line:1000601:LOC', 'line_...",POINT (531434.796 5671771.274),TDG


In [10]:
from src.processors.c2c import C2CBusStopsProcessor

In [11]:
c2c_stops_gdf = C2CBusStopsProcessor.fetch(reload_pipeline=False)
c2c_stops_gdf.loc[:, "source"] = "C2C"
c2c_stops_gdf.head()

,gtfs_id,navitia_id,osm_id,name,description,line_gtfs_ids,line_osm_ids,network,network_gtfs_id,geometry,other,source
0,None,stop_area:OGE:GEN15846,None,"Le Haut-Bréda, Pinsot le Village (Le Haut-Bréda)",None,[],[],Mobilités M - TouGo,None,POINT (679047.781 5677868.337),"{'srid': 3857, 'stoparea_id_and_line': [{'line...",C2C
1,None,stop_area:OGE:GEN15852,None,"Le Haut-Bréda, Hot Pic Belle Etoile (Le Haut-B...",None,[],[],Mobilités M - TouGo,None,POINT (678910.858 5678497.283),"{'srid': 3857, 'stoparea_id_and_line': [{'line...",C2C
2,None,stop_area:OGE:GEN15850,None,"Le Haut-Bréda, Chinfert (Le Haut-Bréda)",None,[],[],Mobilités M - TouGo,None,POINT (678788.406 5679655.482),"{'srid': 3857, 'stoparea_id_and_line': [{'line...",C2C
3,None,stop_area:OGE:GEN15080,None,"Le Haut-Bréda, la Piat (Le Haut-Bréda)",None,[],[],Mobilités M - TouGo,None,POINT (678378.751 5675153.444),"{'srid': 3857, 'stoparea_id_and_line': [{'line...",C2C
4,None,stop_area:OGE:GEN13054,None,"Domène, Domène Mairie (Domène)",None,[],[],Mobilités M - Tag,None,POINT (649900.998 5653440.123),"{'srid': 3857, 'stoparea_id_and_line': [{'line...",C2C


In [12]:
all_stops_gdf = pd.concat(
    (osm_stops_gdf, tdg_stops_gdf.to_crs(EPSG_WGS84), c2c_stops_gdf.to_crs(EPSG_WGS84))
)
all_stops_gdf.tail()

/tmp/ipykernel_15682/3211379376.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_stops_gdf = pd.concat(


,gtfs_id,navitia_id,osm_id,name,description,line_gtfs_ids,line_osm_ids,network,network_gtfs_id,geometry,other,source,stop_code
645,None,stop_area:O38:3226468,NaN,École (Auris),None,[],[],Isère - Transisère,None,POINT (6.0865 45.0459),"{'srid': 3857, 'stoparea_id_and_line': [{'line...",C2C,<NA>
646,None,stop_area:O38:3224622,NaN,Les Égaux (Saint-Pierre-de-Chartreuse),None,[],[],Isère - Transisère,None,POINT (5.80145 45.31654),"{'srid': 3857, 'stoparea_id_and_line': [{'line...",C2C,<NA>
647,None,stop_area:O38:3224623,NaN,Gérentière (Saint-Pierre-de-Chartreuse),None,[],[],Isère - Transisère,None,POINT (5.80912 45.31962),"{'srid': 3857, 'stoparea_id_and_line': [{'line...",C2C,<NA>
648,None,stop_area:O38:3224627,NaN,École de Saint-Hugues (Saint-Pierre-de-Chartre...,None,[],[],Isère - Transisère,None,POINT (5.80637 45.32302),"{'srid': 3857, 'stoparea_id_and_line': [{'line...",C2C,<NA>
649,None,stop_area:O38:3224625,NaN,Saint-Hugues (Saint-Pierre-de-Chartreuse),None,[],[],Isère - Transisère,None,POINT (5.80648 45.32368),"{'srid': 3857, 'stoparea_id_and_line': [{'line...",C2C,<NA>


## Analyse data

In [13]:
interesting_columns = [
    "Id wp",
    "Name wp",
    "nbr_topo",
    "geometry",
    "osm_id",
    "gtfs_id",
    "name",
    "description",
    "line_osm_ids",
    "line_gtfs_ids",
    "network",
    "distance",
]
activity_stops_gdf = gpd.sjoin_nearest(
    isere_activity_gdf.to_crs(EPSG_WEB_MERCATOR),
    all_stops_gdf.to_crs(EPSG_WEB_MERCATOR),
    how="left",
    distance_col="distance",
)
activity_stops_gdf[interesting_columns].sort_values("distance")

,Id wp,Name wp,nbr_topo,geometry,osm_id,gtfs_id,name,description,line_osm_ids,line_gtfs_ids,network,distance
196,145472,Corrençon - Foyer de ski de fond,9,POINT (614267.184 5625414.299),4.931381e+07,None,Hauts Plateaux,None,"[3044780, 19169348, 19169352]",[],Cars Région Isère,0.868801
296,218679,Claix,4,POINT (632695.881 5642081.812),1.803020e+09,None,Les Cimentiers,None,"[3931670, 3931671, 18998238, 18998239, 1947519...",[],M réso,1.671476
294,212026,Gresse en Vercors,3,POINT (619739.873 5606131.081),NaN,None,Le Village (Gresse-en-Vercors),None,[],[],Isère - Transisère,2.136405
383,498794,Les Cloîtres,1,POINT (651692.463 5682037.155),NaN,None,Les Cloitres (Saint-Pierre-dEntremont),None,[],[],Isère - Transisère,2.208040
342,280466,Col de la Charmette,4,POINT (638983.785 5672336.78),1.238612e+10,None,Col de la Charmette,None,"[18352925, 18352926]",[],M réso,2.597828
...,...,...,...,...,...,...,...,...,...,...,...,...
411,680281,Cascade des 7 Laux,8,POINT (674801.61 5654708.703),1.116080e+09,None,Le Collet-Tennis,None,"[7791630, 7791632]",[],Cars Région Isère,7539.995287
71,104267,Clavans le Haut - Hameau du Perron,18,POINT (686251.376 5637271.894),NaN,FR:38237:ZE:6520:ISERE,LA COMBE,<NA>,NaN,"{'line_id': 'ISERE:Line:BDO02:LOC', 'line_name...",REGION - cars Région Isère,7698.097443
267,192778,Pas de Chabrinel,1,POINT (610332.708 5593797.615),NaN,None,Carrefour Richardière (Chichilianne),None,[],[],Isère - Transisère,7861.490821
123,104584,Le Rivier d'Allemont,48,POINT (672418.482 5653549.452),NaN,FR:38527:ZE:1547:ISERE,L'EGLISE,<NA>,NaN,"{'line_id': 'ISERE:Line:BDO04:LOC', 'line_name...",REGION - cars Région Isère,8064.519827


In [14]:
distance_threshold = 5000
too_far_activity_points = activity_stops_gdf[
    activity_stops_gdf["distance"] > distance_threshold
]
print(
    f"{len(too_far_activity_points)} activity points far from any bus stops than {distance_threshold}m"
)

14 activity points far from any bus stops than 5000m


In [15]:
too_far_activity_points.loc[:, "lat"] = (
    too_far_activity_points["geometry"].to_crs(EPSG_WGS84).apply(lambda p: p.y)
)
too_far_activity_points.loc[:, "lon"] = (
    too_far_activity_points["geometry"].to_crs(EPSG_WGS84).apply(lambda p: p.x)
)

/data/hdd/Documents/GitHub/mobilite_durable/.venv/lib/python3.13/site-packages/geopandas/geodataframe.py:1981: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/data/hdd/Documents/GitHub/mobilite_durable/.venv/lib/python3.13/site-packages/geopandas/geodataframe.py:1981: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [16]:
too_far_activity_points_lon_sorted = too_far_activity_points[
    ["lat", "lon", "Name wp"]
].sort_values("lon")
too_far_activity_points_lon_sorted

,lat,lon,Name wp
267,44.823626,5.482712,Pas de Chabrinel
466,44.801365,5.495224,Bergerie du Jas Neuf
498,45.183285,5.965189,Cirque supérieur du Boulon
123,45.203082,6.040438,Le Rivier d'Allemont
131,44.907750,6.054330,Valsenestre
411,45.210419,6.061846,Cascade des 7 Laux
265,45.208139,6.117582,Barrage de Grand Maison
75,45.087469,6.148888,Col de Sarenne
71,45.099960,6.164701,Clavans le Haut - Hameau du Perron
318,45.038969,6.185541,Parking de la Pisse


In [17]:
too_far_activity_points_lon_sorted[2:]["lat"].min()

np.float64(44.907749999995936)

In [18]:
too_far_activity_points_lon_sorted[2:]["lat"].max()

np.float64(45.21041899999998)

In [19]:
min_lon_box = 5.9
max_lon_box = 6.31
min_lat_box = 44.9
max_lat_box = 45.3
sub_map_box = box(min_lon_box, min_lat_box, max_lon_box, max_lat_box)

In [20]:
sub_map_activity_gdf = isere_activity_gdf.cx[min_lon_box:max_lon_box, min_lat_box:max_lat_box]
print(
    f"Filter Isère activities on sub map: keep {len(sub_map_activity_gdf)} on {len(isere_activity_gdf)}"
)

Filter Isère activities on sub map: keep 122 on 514


In [21]:
sub_map_all_stops_gdf = all_stops_gdf.cx[min_lon_box:max_lon_box, min_lat_box:max_lat_box]
print(f"Filter bus stops on sub map: keep {len(sub_map_all_stops_gdf)} on {len(all_stops_gdf)}")

Filter bus stops on sub map: keep 990 on 22482


# Visualize

## All too far activity points

In [22]:
# Plot points with bus data far from distance_threshold m

# Add a base OSM map centered on Grenoble
m = folium.Map(location=[45.1885, 5.7245], zoom_start=9, tiles="OpenStreetMap")

# Add Waymarked Trails hiking layers
folium.TileLayer(tiles="WaymarkedTrails.hiking").add_to(m)

# Add Isère polygon
isere_latlon_coords = [
    (lat, lon)
    for lon, lat in area_gdf[area_gdf["code"] == isere_code]["geometry"]
    .values[0]
    .exterior.coords
]
folium.Polygon(
    locations=isere_latlon_coords,
    color="black",
    weight=3,
    fill_color="black",
    fill_opacity=0.25,
    fill=True,
).add_to(m)

# Add activity markers
columns_to_display = ["Id wp", "Name wp", "nbr_topo"]
folium.GeoJson(
    too_far_activity_points[columns_to_display + ["geometry"]],
    zoom_on_click=True,
    marker=folium.Marker(icon=folium.Icon(icon="star", color="purple")),
    tooltip=folium.GeoJsonTooltip(fields=columns_to_display),
    popup=folium.GeoJsonPopup(fields=columns_to_display),
).add_to(m)
m

In [23]:
m.save("no_reachable_activity_points.html")

## All data on submap

In [24]:
# Copy dataframe and add color
activity_columns_to_display = ["Id wp", "Name wp", "nbr_topo"]
sub_map_activity_gdf_to_plot = sub_map_activity_gdf[
    activity_columns_to_display + ["geometry"]
].copy()
sub_map_activity_gdf_to_plot["color"] = "green"
sub_map_activity_gdf_to_plot.loc[
    sub_map_activity_gdf_to_plot["Id wp"].isin(too_far_activity_points["Id wp"]), "color"
] = "purple"
sub_map_activity_gdf_to_plot.head()

,Id wp,Name wp,nbr_topo,geometry,color
1,39041,Alpe d'Huez - Falaises du lac Besson,5,POINT (6.09444 45.11727),green
2,39108,Les Arias d'en bas (Mariande),1,POINT (6.17616 44.91703),green
3,39178,Cirque inférieur du Boulon,1,POINT (5.96136 45.19128),green
4,39195,Le Bourg-d'Oisans - Commères,11,POINT (6.08117 45.0272),green
6,39918,Le Vénéon - Vallon de la Selle (ou du Diable),31,POINT (6.17787 44.96261),green


In [25]:
# Plot points with bus data far from distance_threshold m

# Add a base OSM map centered on the box
m = folium.Map(
    location=[(min_lat_box + max_lat_box) / 2, (min_lon_box + max_lon_box) / 2],
    zoom_start=10,
    tiles="OpenStreetMap",
)

# Add Waymarked Trails hiking layers
folium.TileLayer(tiles="WaymarkedTrails.hiking", control=False).add_to(m)

# Add Isère polygon interscetion with box
isere_latlon_coords = [
    (lat, lon)
    for lon, lat in area_gdf[area_gdf["code"] == isere_code]["geometry"]
    .values[0]
    .intersection(sub_map_box)
    .exterior.coords
]
folium.Polygon(
    locations=isere_latlon_coords,
    color="black",
    weight=3,
    fill_color="black",
    fill_opacity=0.25,
    fill=True,
).add_to(m)

# Add bus markers
source_to_id = {"TDG": "gtfs_id", "OSM": "osm_id", "C2C": "navitia_id"}
for source in sub_map_all_stops_gdf["source"].unique():
    fg = folium.FeatureGroup(name=f"Bus stops from {source}", show=True)
    fields = [source_to_id[source], "name", "network"]
    aliases = [f"<b>Arrêt de bus de {source}</b><br>ID :", "Nom :", "Réseau :"]
    folium.GeoJson(
        sub_map_all_stops_gdf[sub_map_all_stops_gdf["source"] == source][fields + ["geometry"]],
        zoom_on_click=True,
        marker=folium.Marker(icon=folium.Icon(icon="bus", prefix="fa", color="blue")),
        tooltip=folium.GeoJsonTooltip(fields=fields, aliases=aliases),
        popup=folium.GeoJsonPopup(fields=fields, aliases=aliases),
    ).add_to(fg)
    fg.add_to(m)


# Add activity markers
fg = folium.FeatureGroup(name="C2C activity access point", show=True)
aliases = [f"<b>Point d'accès d'activité</b><br>ID :", "Nom :", "Nombre de topos :"]
folium.GeoJson(
    sub_map_activity_gdf_to_plot,
    zoom_on_click=True,
    marker=folium.Marker(icon=folium.Icon(icon="star")),
    tooltip=folium.GeoJsonTooltip(fields=activity_columns_to_display, aliases=aliases),
    popup=folium.GeoJsonPopup(fields=activity_columns_to_display, aliases=aliases),
    style_function=lambda x: {
        "markerColor": x["properties"]["color"],
    },
).add_to(fg)
fg.add_to(m)

# Add layer control to filter layers
folium.LayerControl(collapsed=False).add_to(m)

m

In [26]:
m.save("submap.html")

# Analyse & visualise - C2C only bus stop

In [27]:
c2c_only_interesting_columns = [
    "Id wp",
    "Name wp",
    "nbr_topo",
    "geometry",
    "name",
    "network",
    "distance",
]
c2c_only_activity_stops_gdf = gpd.sjoin_nearest(
    isere_activity_gdf.to_crs(EPSG_WEB_MERCATOR),
    c2c_stops_gdf.to_crs(EPSG_WEB_MERCATOR),
    how="left",
    distance_col="distance",
)
c2c_only_activity_stops_gdf[c2c_only_interesting_columns].sort_values("distance")

,Id wp,Name wp,nbr_topo,geometry,name,network,distance
294,212026,Gresse en Vercors,3,POINT (619739.873 5606131.081),Le Village (Gresse-en-Vercors),Isère - Transisère,2.136405
383,498794,Les Cloîtres,1,POINT (651692.463 5682037.155),Les Cloitres (Saint-Pierre-dEntremont),Isère - Transisère,2.208040
296,218679,Claix,4,POINT (632695.881 5642081.812),"Claix, les Cimentiers (Claix)",Mobilités M - Tag,4.778016
467,1217355,Theys - centre village,1,POINT (667492.918 5668911.073),"Theys, Theys Écoles (Theys)",Mobilités M - TouGo,4.810536
357,338704,Montfort Funiculaire,2,POINT (656689.261 5668649.282),"Lumbin, Montfort - Funiculaire (Lumbin)",Mobilités M - Tag,5.122381
...,...,...,...,...,...,...,...
3,39178,Cirque inférieur du Boulon,1,POINT (663615.448 5651685.199),La Ville (Allemond),Isère - Transisère,12962.114335
351,315696,Dionay,1,POINT (580982.656 5653574.099),Le Village (Cognin-les-Gorges),Isère - Transisère,22081.252975
513,1778130,Eydoche,1,POINT (593320.831 5691278.95),Barbusse Gare Routière (Bourgoin-Jallieu),CAPI - Ruban,24862.726000
513,1778130,Eydoche,1,POINT (593320.831 5691278.95),Barbusse Gare Routière (Bourgoin-Jallieu),Isère - Transisère,24862.726000


In [28]:
c2c_only_activity_stops_gdf[c2c_only_interesting_columns].sort_values("distance").to_csv(
    "nearest_c2c_bus_stop.csv", index=None
)

In [29]:
c2c_only_too_far_activity_points = c2c_only_activity_stops_gdf[
    c2c_only_activity_stops_gdf["distance"] > distance_threshold
]
print(
    f"{len(c2c_only_too_far_activity_points)} activity points far from any C2C bus stops than {distance_threshold}m"
)

64 activity points far from any C2C bus stops than 5000m


In [30]:
# Plot points with bus data far from distance_threshold m

# Add a base OSM map centered on Grenoble
m = folium.Map(location=[45.1885, 5.7245], zoom_start=9, tiles="OpenStreetMap")

# Add Waymarked Trails hiking layers
folium.TileLayer(tiles="WaymarkedTrails.hiking").add_to(m)

columns_to_display = ["Id wp", "Name wp", "nbr_topo"]
folium.GeoJson(
    c2c_only_too_far_activity_points[columns_to_display + ["geometry"]],
    name="C2C stop areas not merged with Transport Data Gouv",
    zoom_on_click=True,
    marker=folium.Marker(icon=folium.Icon(icon="star", color="purple")),
    tooltip=folium.GeoJsonTooltip(fields=columns_to_display),
    popup=folium.GeoJsonPopup(fields=columns_to_display),
).add_to(m)
m

In [31]:
m.save("c2c_no_reachable_activity_points.html")